In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime, timezone
from my_apis.mb_connection import MetabaseConnection
from my_apis.sheets_functions import SheetsFunctions
from my_apis.sf_connection import SalesforceConnection, SalesforceFunctions

In [2]:
sf_credentials = '~/.keys/sf_credentials.json'
mb_credentials = '~/.keys/mb_credentials.json'

sfc = SalesforceConnection(os.path.expanduser(sf_credentials))
mbc = MetabaseConnection(os.path.expanduser(mb_credentials))
sff = SalesforceFunctions(sfc)

In [8]:
new_entry = pd.read_excel('templates/auxiliar_new_entry.xlsx')[['mp_sf_id', 'category', 'subcategory']]

In [4]:
# 0. Sacamos todos los productos de manufactura
manufacturing_product_type = sff.get_record_type_id('Product2', 'Manufacturing')

query = f'''
select Id, Name, manufacturing_product_category__c
from Product2
where RecordTypeId = '{manufacturing_product_type}'
'''
products = sfc.extract_data(query)
manufacturing_product_type

'012TP000000U1zZYAS'

In [10]:
# 1. sacar los productos que tiene asignados el mp
mps_ids = tuple(new_entry.mp_sf_id.unique().tolist())

if len(mps_ids) == 1:
    where_clause = f"where account__c = '{mps_ids[0]}'"
else:
    where_clause = f'where account__c in {mps_ids}'
    

query = f'''
select account__c, category__c, subcategory__c, product__c
from manufacturing_product__c
{where_clause}
'''

try:
    current_products = sfc.extract_data(query)
except KeyError:
    current_products = pd.DataFrame(columns=['account__c', 'subcategory__c'])

In [11]:
# 3. encontramos los productos que estén en new_entry y que no estén en current_products
(
    new_entry
    .merge(current_products, left_on=['subcategory', 'mp_sf_id'], right_on=['subcategory__c', 'account__c'], how='left')
    .query('account__c.isna()')
    .merge(products, left_on='subcategory', right_on='Name')
    [['mp_sf_id', 'Id']]
    .rename({'mp_sf_id':'account__c', 'Id':'product__c'}, axis=1)
    .pipe(lambda df: sff.add_multiple_records(df, 'manufacturing_product__c'))
)

[]